<a href="https://colab.research.google.com/github/suparuek2405/Stock-screening-with-python/blob/main/Group_assignment_Tacocat_set_yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install libraries
# !pip install pandas
# !pip install numpy
# !pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
#Function for get data from yfinance
def rename_columns_to_year_end(df):
    df_rt = df.copy()
    years = df_rt.columns.year
    year_end_dates = [pd.Timestamp(f"{year}-12-31") for year in years]
    df_rt.columns = year_end_dates
    return df_rt

def get_balance_sheet(ticker):
  balance_sheet_df = pd.DataFrame()
  n = len(ticker)
  print("start getting balance sheet")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.balancesheet

    #rename column
    bs = rename_columns_to_year_end(raw)

      #reset index and rename it
    bs.reset_index(inplace=True)
    bs.rename(columns={'index': 'data'}, inplace=True)
    bs['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    bs = bs[~bs.duplicated(subset=['data'])]
    bs.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    bs.columns = pd.to_datetime(bs.columns)

    #append to dataframe
    balance_sheet_df = pd.concat([balance_sheet_df, bs])
    print(f"{c+1}/{n} {i}")
  print("finish getting balance sheet")
  return balance_sheet_df

def get_income_statement(ticker):
  income_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting income statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.financials

    #rename column
    is_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    is_df.reset_index(inplace=True)
    is_df.rename(columns={'index': 'data'}, inplace=True)
    is_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    is_df = is_df[~is_df.duplicated(subset=['data'])]
    is_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    is_df.columns = pd.to_datetime(is_df.columns)

    #append to dataframe
    income_statement_df = pd.concat([income_statement_df, is_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting income statement")
  return income_statement_df

def get_cashflow_statement(ticker):
  cashflow_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting cashflow statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.cashflow

    #rename column
    cf_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    cf_df.reset_index(inplace=True)
    cf_df.rename(columns={'index': 'data'}, inplace=True)
    cf_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    cf_df = cf_df[~cf_df.duplicated(subset=['data'])]
    cf_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    cf_df.columns = pd.to_datetime(cf_df.columns)

    #append to dataframe
    cashflow_statement_df = pd.concat([cashflow_statement_df, cf_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting cashflow statement")
  return cashflow_statement_df

def get_monthly_stock_prices(ticker,start_date = "2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  data = stock.history(start=start_date, end=end_date, interval="1mo")
  data = data.set_index(pd.to_datetime(data.index))

  # Resample to the start of each month
  monthly_prices = data['Close'].resample('MS').first()
  monthly_prices = pd.DataFrame(monthly_prices)
  monthly_prices['symbol'] = ticker.split('.')[0]
  monthly_prices.index = monthly_prices.index.to_period('M').to_timestamp(how='start')
  return monthly_prices

def get_dividend_data(ticker, start_date="2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  dividends = stock.dividends
  # Filter dividends based on the specified date range
  dividends = dividends[(dividends.index >= start_date) & (dividends.index <= end_date)]
  if not dividends.empty:
    dividends_df = pd.DataFrame(dividends)
    dividends_df['symbol'] = ticker.split('.')[0]
    dividends_df = dividends_df.rename(columns={'Dividends':'amount'})
    dividends_df.index = dividends_df.index.to_period('M').to_timestamp(how='start')
    return dividends_df

def get_financial_data(ticker):
  balance_sheet = get_balance_sheet(ticker)
  income_statement = get_income_statement(ticker)
  cashflow_statement = get_cashflow_statement(ticker)
  return balance_sheet, income_statement, cashflow_statement

In [ ]:
#df = pd.read_csv('/content/drive/My Drive/Tools/Group Assignment/set_scraped.csv')
df = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/main/set_scraped.csv')

In [ ]:
bkk = df.copy()
bkk = bkk[['symbol', 'text-start', 'text-start 2', 'symbol 2', 'symbol 3']]
bkk.columns = ['symbol', 'company_name', 'market', 'industry', 'sector']
bkk['ticker'] = bkk['symbol'] + '.BK'
bkk.head(10)

,symbol,company_name,market,industry,sector,ticker
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,NaN,24CS.BK
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,2S.BK
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,3K-BAT.BK
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,A.BK
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,NaN,A5.BK
5,AAI,บริษัท เอเชี่ยน อะไลอันซ์ อินเตอร์เนชั่นแนล จำ...,SET,AGRO,FOOD,AAI.BK
6,AAV,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน),SET,SERVICE,TRANS,AAV.BK
7,ABM,บริษัท เอเชีย ไบโอแมส จำกัด (มหาชน),mai,RESOURC,NaN,ABM.BK
8,ACAP,บริษัท เอเชีย แคปปิตอล กรุ๊ป จำกัด (มหาชน),mai,FINCIAL,NaN,ACAP.BK
9,ACC,บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...,SET,RESOURC,ENERG,ACC.BK


In [ ]:
set_list = bkk['ticker']
ticker_error = ['DIMET.BK'] #exclude error ticker
set_list = set_list[set_list.isin(ticker_error) == False]

In [ ]:
#run this code to start retrive data from yfinance
bl_st, ic_st, cf_st = get_financial_data(set_list)

In [ ]:
# run this code below to download file
# bl_st.to_csv('balance_sheet.csv')
# ic_st.to_csv('income_statement.csv')
# cf_st.to_csv('cashflow_statement.csv')

# from google.colab import files
# files.download('balance_sheet.csv')
# files.download('income_statement.csv')
# files.download('cashflow_statement.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------

## After get data


In [212]:
#do not change variable name due to function using this as global variable
df = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/main/set_scraped.csv')
bkk = df.copy()
bl = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/balance_sheet.csv')
ic = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/income_statement.csv')
cf = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/cashflow_statement.csv')

In [ ]:
bkk

,symbol,company_name,market,industry,sector,ticker
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,NaN,24CS.BK
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,2S.BK
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,3K-BAT.BK
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,A.BK
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,NaN,A5.BK
...,...,...,...,...,...,...
907,YONG,บริษัท ยงคอนกรีต จำกัด (มหาชน),mai,PROPCON,NaN,YONG.BK
908,YUASA,บริษัท ยัวซ่าแบตเตอรี่ ประเทศไทย จำกัด (มหาชน),mai,INDUS,NaN,YUASA.BK
909,ZAA,บริษัท ซาเล็คต้า จำกัด (มหาชน),SET,SERVICE,MEDIA,ZAA.BK
910,ZEN,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน),SET,AGRO,FOOD,ZEN.BK


In [ ]:
bl.data.unique()

array(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value',
       'Invested Capital', 'Working Capital', 'Net Tangible Assets',
       'Capital Lease Obligations', 'Common Stock Equity',
       'Total Capitalization', 'Total Equity Gross Minority Interest',
       'Stockholders Equity', 'Other Equity Interest',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Employee Benefits',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current 

In [ ]:
ic.data.unique()

array(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense',
       'Interest Income', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision',
       'Pretax Income', 'Other Income Expense',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Gain On Sale Of Ppe', 'Other Special Charges',
       'Gain On Sale Of Security'

In [ ]:
cf.data.unique()

array(['Free Cash Flow', 'Repayment Of Debt', 'Issuance Of Debt',
       'Issuance Of Capital Stock', 'Capital Expenditure',
       'End Cash Position', 'Other Cash Adjustment Outside Changein Cash',
       'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow',
       'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Issuance', 'Net Issuance Payments Of Debt',
       'Net Short Term Debt Issuance', 'Short Term Debt Payments',
       'Short Term Debt Issuance', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow',
       'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Interest Received Cfi',
       'Net Business Purchase And Sale', 'Purchase Of Business',
       'Net Intangibles Purchase And Sale', 'Purchase Of In

### Function for Growth investment scanning

2. Growth Investing Using Return on Equity (ROE) and Earnings Growth
* Return on Equity (ROE): Focus on companies with high and increasing ROE, ideally above 15%. (เช็คความต่อเนื่องของ ROE ด้วยว่าคงที่หรือไม่ บางธุรกิจที่เป็น commodity จะมีค่า ROE สูงเว่อร์บ้างช่วงได้)
* Revenue Growth (Income Statement): Look for consistent revenue growth (at least 10% year-over-year) to ensure the company is expanding its market presence.
* Net Profit Margin (Income Statement): A high and improving profit margin (above 10%) shows the company is efficient at turning revenue into profit.
* Return on Assets (ROA) (Income Statement & Balance Sheet): ROA measures profitability relative to total assets. A higher ROA (above 5%) indicates efficient use of assets to generate earnings.
* Operating Cash Flow Growth (Cash Flow Statement): Look for companies with consistent operating cash flow growth. Growing cash flow ensures that earnings growth is supported by actual cash generation.
* Debt-to-Equity (D/E) Ratio (Balance Sheet): For growth companies, a manageable D/E ratio is important. A ratio under 1 is preferable, showing the company is not overly dependent on debt.
* Objective: Focus on companies that not only have strong profitability metrics like ROE but also show consistent revenue, earnings, and cash flow growth to support long-term expansion. (อาจดูเรื่องของ CAPEX เทียบ ยอดขายเพื่อดูสัดส่วนการลงทุนเพื่อการเติบโตของบริษัท)>> Rank by ROE

In [206]:
#Function for Growth investment scanning
from datetime import datetime

def get_lastdateofyear(start_year, end_year):
    """
    Generate the last date of each year between the start year and end year, inclusive.

    Args:
        start_year (int): The starting year.
        end_year (int): The ending year.

    Returns:
        list: A list of strings, where each string represents the last date of the year (in 'YYYY-MM-DD' format)
              for every year from start_year to end_year, inclusive.
    """
    last_dates = []

    for year in range(start_year, end_year + 1):
        last_date = datetime(year, 12, 31).strftime('%Y-%m-%d')
        last_dates.append(last_date)

    return last_dates

def get_roe(startyear=2020, endyear=2022):
  """
    Calculate the Return on Equity (ROE) for each company symbol for the given years.

    The function retrieves 'Net Income' and 'Stockholders' Equity' from the datasets (`ic` and `bl` respectively),
    merges the data on the 'symbol' column, and calculates ROE for each year as:

    ROE = Net Income / Stockholders' Equity

    Args:
        startyear (int, optional): The starting year for which to calculate ROE. Default is 2020.
        endyear (int, optional): The ending year for which to calculate ROE. Default is 2022.

    Returns:
        pandas.DataFrame: A DataFrame containing the company 'symbol', 'ROE' ratio for each year between startyear
                          and endyear, and an additional 'ratio' column that is set to "ROE".
                          The column structure will be ['symbol', 'ratio', year1, year2, ..., yearN].

    Example:
        >>> df = get_roe(2020, 2022)
        >>> print(df)
          symbol ratio  2020-12-31  2021-12-31  2022-12-31
        0   AAPL   ROE        0.15        0.18        0.20
        1   MSFT   ROE        0.12        0.13        0.14
    """
  #get lastdate of year to retrive
  col = get_lastdateofyear(startyear, endyear)

  #create copy of data to calculate
  net_income = ic[ic.data == 'Net Income'][['symbol', 'data'] + col].copy()
  stockholder_equity = bl[bl.data == 'Stockholders Equity'][['symbol', 'data'] + col].copy()

  #merge table
  df_merge = pd.merge(net_income, stockholder_equity, on='symbol', suffixes=('_ni', '_se'))

  # Calculate ROE: Net Income / Average Stockholders' Equity
  roe_dict = {}
  for year in col:
      # Calculate ROE: Net Income / Stockholders' Equity
      df_merge[f'{year}'] = df_merge[f'{year}_ni'] / df_merge[f'{year}_se']

      # Store the ROE results for the year in the dictionary
      roe_dict[f'{year}'] = df_merge[[f'{year}']]

  #prep return dataframe
  df = df_merge[['symbol'] + [f'{year}' for year in col]].copy()
  df['ratio'] = "ROE"

  return df[['symbol', 'ratio'] + col]

def get_revenue_growth_yoy(startyear=2020, endyear=2022):
  """
  Calculate the year-over-year revenue growth rate for each company for the specified years.

  Args:
    startyear: The starting year for the calculation (inclusive).
    endyear: The ending year for the calculation (inclusive).

  Returns:
    A Pandas DataFrame containing the 'symbol' and 'Revenue Growth' for each year.
  """

  col = get_lastdateofyear(startyear, endyear)
  revenue = ic[ic.data == 'Total Revenue'][['symbol', 'data'] + col].copy()

  revenue_growth_dict = {}
  for i in range(len(col) - 1):
    current_year = col[i + 1]
    previous_year = col[i]
    revenue[f'{current_year}_growth'] = (revenue[current_year] - revenue[previous_year]) / revenue[previous_year]

    # Store the revenue growth results for the year in the dictionary
    revenue_growth_dict[f'{current_year}_growth'] = revenue[[f'{current_year}_growth']]

  # Prepare the return DataFrame
  df = revenue[['symbol'] + [f'{year}_growth' for year in col[1:]]].copy()
  df['ratio'] = 'Revenue Growth YOY'

  # Rename columns by removing the '_growth' suffix
  df.rename(columns={f'{year}_growth': f'{year}' for year in col[1:]}, inplace=True)

  return df[['symbol', 'ratio'] + [f'{year}' for year in col[1:]]]

def get_net_profit_margin(startyear=2020, endyear=2022):
  """
    Calculate the Net Profit Margin for each company symbol for the given years.

    The function retrieves 'Net Income' and 'Total Revenue' from the datasets (`ic`),
    merges the data on the 'symbol' column, and calculates Net Profit Margin for each year as:

    Net Profit Margin = Net Income / Total Revenue

    Args:
        startyear (int, optional): The starting year for which to calculate Net Profit Margin. Default is 2020.
        endyear (int, optional): The ending year for which to calculate Net Profit Margin. Default is 2022.

    Returns:
        pandas.DataFrame: A DataFrame containing the company 'symbol', 'Net Profit Margin' ratio for each year between startyear
                          and endyear, and an additional 'ratio' column that is set to "Net Profit Margin".
                          The column structure will be ['symbol', 'ratio', year1, year2, ..., yearN].

    Example:
        >>> df = get_net_profit_margin(2020, 2022)
        >>> print(df)
          symbol ratio  2020-12-31  2021-12-31  2022-12-31
        0   AAPL   Net Profit Margin        0.15        0.18        0.20
        1   MSFT   Net Profit Margin        0.12        0.13        0.14
    """
  #get lastdate of year to retrive
  col = get_lastdateofyear(startyear, endyear)

  #create copy of data to calculate
  net_income = ic[ic.data == 'Net Income'][['symbol', 'data'] + col].copy()
  total_revenue = ic[ic.data == 'Total Revenue'][['symbol', 'data'] + col].copy()

  #merge table
  df_merge = pd.merge(net_income, total_revenue, on='symbol', suffixes=('_ni', '_tr'))

  # Calculate Net Profit Margin: Net Income / Total Revenue
  npm_dict = {}
  for year in col:
      # Calculate Net Profit Margin: Net Income / Total Revenue
      df_merge[f'{year}'] = df_merge[f'{year}_ni'] / df_merge[f'{year}_tr']

      # Store the Net Profit Margin results for the year in the dictionary
      npm_dict[f'{year}'] = df_merge[[f'{year}']]

  #prep return dataframe
  df = df_merge[['symbol'] + [f'{year}' for year in col]].copy()
  df['ratio'] = "Net Profit Margin"

  return df[['symbol', 'ratio'] + col]

def get_roa(startyear=2020, endyear=2022):
  """
    Calculate the Return on Assets (ROA) for each company symbol for the given years.

    The function retrieves 'Net Income' from the income statement dataset (`ic`) and 'Total Assets'
    from the balance sheet dataset (`bl`), merges the data on the 'symbol' column, and calculates ROA
    for each year as:

    ROA = Net Income / Total Assets

    Args:
        startyear (int, optional): The starting year for which to calculate ROA. Default is 2020.
        endyear (int, optional): The ending year for which to calculate ROA. Default is 2022.

    Returns:
        pandas.DataFrame: A DataFrame containing the company 'symbol', 'ROA' ratio for each year between startyear
                          and endyear, and an additional 'ratio' column that is set to "ROA".
                          The column structure will be ['symbol', 'ratio', year1, year2, ..., yearN].

    Example:
        >>> df = get_roa(2020, 2022)
        >>> print(df)
          symbol ratio  2020-12-31  2021-12-31  2022-12-31
        0   AAPL   ROA        0.08        0.10        0.12
        1   MSFT   ROA        0.06        0.07        0.08
    """
  # Get the last date of each year
  col = get_lastdateofyear(startyear, endyear)

  # Create copies of the dataframes for calculation
  net_income = ic[ic.data == 'Net Income'][['symbol', 'data'] + col].copy()
  total_assets = bl[bl.data == 'Total Assets'][['symbol', 'data'] + col].copy()

  # Merge the two dataframes on the 'symbol' column
  df_merge = pd.merge(net_income, total_assets, on='symbol', suffixes=('_ni', '_ta'))

  # Calculate ROA for each year and store the results in a dictionary
  roa_dict = {}
  for year in col:
    # Calculate ROA: Net Income / Total Assets
    df_merge[f'{year}'] = df_merge[f'{year}_ni'] / df_merge[f'{year}_ta']

    # Store the ROA results for the year in the dictionary
    roa_dict[f'{year}'] = df_merge[[f'{year}']]

  # Prepare the return dataframe
  df = df_merge[['symbol'] + [f'{year}' for year in col]].copy()
  df['ratio'] = "ROA"

  return df[['symbol', 'ratio'] + col]

def get_operating_cashflow_growth(startyear=2020, endyear=2022):
  """
  Calculate the year-over-year operating cash flow growth rate for each company for the specified years.

  Args:
    startyear: The starting year for the calculation (inclusive).
    endyear: The ending year for the calculation (inclusive).

  Returns:
    A Pandas DataFrame containing the 'symbol' and 'Operating Cash Flow Growth' for each year.
  """

  col = get_lastdateofyear(startyear, endyear)
  operating_cashflow = cf[cf.data == 'Operating Cash Flow'][['symbol', 'data'] + col].copy()

  ocf_growth_dict = {}
  for i in range(len(col) - 1):
    current_year = col[i + 1]
    previous_year = col[i]
    operating_cashflow[f'{current_year}_growth'] = (operating_cashflow[current_year] - operating_cashflow[previous_year]) / operating_cashflow[previous_year]

    # Store the operating cash flow growth results for the year in the dictionary
    ocf_growth_dict[f'{current_year}_growth'] = operating_cashflow[[f'{current_year}_growth']]

  # Prepare the return DataFrame
  df = operating_cashflow[['symbol'] + [f'{year}_growth' for year in col[1:]]].copy()
  df['ratio'] = 'Operating Cash Flow Growth YOY'

  # Rename columns by removing the '_growth' suffix
  df.rename(columns={f'{year}_growth': f'{year}' for year in col[1:]}, inplace=True)

  return df[['symbol', 'ratio'] + [f'{year}' for year in col[1:]]]

def get_de(startyear=2020, endyear=2022):
  """
    Calculate the Debt-to-Equity (D/E) Ratio for each company symbol for the given years.

    The function retrieves 'Total Debt' and 'Stockholders' Equity' from the datasets (`bl`),
    merges the data on the 'symbol' column, and calculates D/E Ratio for each year as:

    D/E Ratio = Total Debt / Stockholders' Equity

    Args:
        startyear (int, optional): The starting year for which to calculate D/E Ratio. Default is 2020.
        endyear (int, optional): The ending year for which to calculate D/E Ratio. Default is 2022.

    Returns:
        pandas.DataFrame: A DataFrame containing the company 'symbol', 'D/E Ratio' ratio for each year between startyear
                          and endyear, and an additional 'ratio' column that is set to "D/E Ratio".
                          The column structure will be ['symbol', 'ratio', year1, year2, ..., yearN].

    Example:
        >>> df = get_de(2020, 2022)
        >>> print(df)
          symbol ratio  2020-12-31  2021-12-31  2022-12-31
        0   AAPL   D/E Ratio        0.50        0.60        0.70
        1   MSFT   D/E Ratio        0.40        0.50        0.60
    """
  # Get the last date of each year
  col = get_lastdateofyear(startyear, endyear)

  # Create copies of the dataframes for calculation
  total_debt = bl[bl.data == 'Total Debt'][['symbol', 'data'] + col].copy()
  stockholder_equity = bl[bl.data == 'Stockholders Equity'][['symbol', 'data'] + col].copy()

  # Merge the two dataframes on the 'symbol' column
  df_merge = pd.merge(total_debt, stockholder_equity, on='symbol', suffixes=('_td', '_se'))

  # Calculate D/E Ratio for each year and store the results in a dictionary
  de_dict = {}
  for year in col:
    # Calculate D/E Ratio: Total Debt / Stockholders' Equity
    df_merge[f'{year}'] = df_merge[f'{year}_td'] / df_merge[f'{year}_se']

    # Store the D/E Ratio results for the year in the dictionary
    de_dict[f'{year}'] = df_merge[[f'{year}']]

  # Prepare the return dataframe
  df = df_merge[['symbol'] + [f'{year}' for year in col]].copy()
  df['ratio'] = "D/E Ratio"

  return df[['symbol', 'ratio'] + col]

In [207]:
get_roe(2020,2022)

,symbol,ratio,2020-12-31,2021-12-31,2022-12-31
0,24CS,ROE,0.178040,0.142030,0.040216
1,2S,ROE,0.233382,0.323577,-0.060549
2,3K-BAT,ROE,NaN,0.069426,NaN
3,A,ROE,0.054524,-0.111162,0.041458
4,A5,ROE,0.084415,0.161422,0.109213
...,...,...,...,...,...
886,YONG,ROE,0.121621,0.159012,0.120806
887,YUASA,ROE,0.178484,0.103877,0.107010
888,ZAA,ROE,-0.141936,-0.047869,0.054440
889,ZEN,ROE,-0.052559,-0.081583,0.120169


In [208]:
get_revenue_growth_yoy(2020,2022)

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,symbol,ratio,2021-12-31,2022-12-31
45,24CS,Revenue Growth YOY,0.597930,0.525501
92,2S,Revenue Growth YOY,0.163802,0.028854
144,3K-BAT,Revenue Growth YOY,0.133534,NaN
198,A,Revenue Growth YOY,-0.438350,0.023125
247,A5,Revenue Growth YOY,-0.095097,-0.114449
...,...,...,...,...
42056,YONG,Revenue Growth YOY,0.089252,0.165335
42103,YUASA,Revenue Growth YOY,0.087654,0.145287
42154,ZAA,Revenue Growth YOY,-0.442723,0.779825
42197,ZEN,Revenue Growth YOY,-0.032942,0.531763


In [209]:
get_net_profit_margin(2020,2022)

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,symbol,ratio,2020-12-31,2021-12-31,2022-12-31
0,24CS,Net Profit Margin,0.017288,0.030330,0.025031
1,2S,Net Profit Margin,0.073204,0.113102,-0.018211
2,3K-BAT,Net Profit Margin,-0.048088,0.028435,NaN
3,A,Net Profit Margin,0.034546,-0.109075,0.041424
4,A5,Net Profit Margin,0.070629,0.178213,0.150577
...,...,...,...,...,...
886,YONG,Net Profit Margin,0.055508,0.079225,0.099581
887,YUASA,Net Profit Margin,0.076409,0.043383,0.042430
888,ZAA,Net Profit Margin,-0.224285,-0.130018,0.087705
889,ZEN,Net Profit Margin,-0.027870,-0.041459,0.045490


In [203]:
get_roa(2020,2022)

,symbol,ratio,2020-12-31,2021-12-31,2022-12-31
0,24CS,ROA,0.048376,0.047231,0.024041
1,2S,ROA,0.186021,0.300108,-0.053445
2,3K-BAT,ROA,NaN,0.038797,NaN
3,A,ROA,0.014825,-0.026823,0.010119
4,A5,ROA,0.030612,0.117907,0.039246
...,...,...,...,...,...
886,YONG,ROA,0.042590,0.062507,0.075735
887,YUASA,ROA,0.112004,0.066475,0.069320
888,ZAA,ROA,-0.085792,-0.030492,0.039884
889,ZEN,ROA,-0.023673,-0.035810,0.054379


In [210]:
get_operating_cashflow_growth(2020,2022)

,symbol,ratio,2021-12-31,2022-12-31
34,24CS,Operating Cash Flow Growth YOY,41.996736,0.851700
89,2S,Operating Cash Flow Growth YOY,0.877883,-0.438364
156,3K-BAT,Operating Cash Flow Growth YOY,-0.993476,NaN
217,A,Operating Cash Flow Growth YOY,-0.641530,-0.916795
280,A5,Operating Cash Flow Growth YOY,0.782828,-2.439997
...,...,...,...,...
53927,YONG,Operating Cash Flow Growth YOY,-0.344102,0.636182
53980,YUASA,Operating Cash Flow Growth YOY,-0.553089,0.559614
54037,ZAA,Operating Cash Flow Growth YOY,0.560627,1.536873
54107,ZEN,Operating Cash Flow Growth YOY,-0.241754,1.179069


In [211]:
get_de(2020,2022)

,symbol,ratio,2020-12-31,2021-12-31,2022-12-31
0,24CS,D/E Ratio,0.629041,0.776858,0.061410
1,2S,D/E Ratio,0.105104,0.002502,0.076428
2,3K-BAT,D/E Ratio,NaN,0.358095,0.317284
3,A,D/E Ratio,2.265847,2.770318,2.723046
4,A5,D/E Ratio,1.191954,0.190751,1.445639
...,...,...,...,...,...
866,YONG,D/E Ratio,1.465588,1.203539,0.427233
867,YUASA,D/E Ratio,0.008164,0.010551,0.009749
868,ZAA,D/E Ratio,0.101844,0.102347,0.016658
869,ZEN,D/E Ratio,0.809678,0.759878,0.670754
